# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [4]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [5]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [6]:
browser = webdriver.Chrome(executable_path='./chromedriver')
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

time.sleep(10)

SCROLL_PAUSE_TIME = 2

# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
last_height = browser.execute_script("return document.body.scrollHeight")
while True:
    print("網頁更新中...")
    # Scroll down to the bottom.
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load the page.
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

print("到達頁面底端")
    

網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
到達頁面底端


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [7]:
# Function連到外部連結，並爬取新聞內容。
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'html.parser')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [8]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "html.parser")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 華邦電Nor Flash出貨營收達歷史新高　產能吃緊延續至下半年

▲華邦電法說會。（圖／翻攝自華邦電法說會直播）
記者周康玉／台北報導
華邦電（2344）今（19）日舉行法說會，記憶體廠華邦電總經理陳沛銘表示，由於目前產能吃緊狀態持續，漲價效應將於第2季、也就是3月、4月顯現，預期DRAM與Flash供給吃緊情況將延續至下半年。
華邦電去年全年營收606.83億元、年增24.4%；毛利率28.08%、年增1.58個百分點；稅後盈餘13.4億元，年增3.8%、每股稅後盈餘0.33元。華邦電去年第4季營收達203.48億元，季增26.9%、年增63.43%；毛利率30.43%，季增3.47個百分點、年增8.3個百分點；稅後盈餘3.5億元，季增5.74%、較前年同期轉虧為盈，每股稅後盈餘0.09元。
陳沛銘表示，去年下半年受疫情影響，第4季淡季不淡，全年營運呈現逐季增長，且因DRAM價格低迷，因此在有限產能下，將產品比重往Flash轉移，Flash和DARM營收占比從第1季的47：53，到第4季比重成為57：43。整體而言，DRAM出貨位元有高十位數（high-teens）的年增率，但全年營收個位數百分比下滑。
陳沛銘表示，由於終端需求望，加上大部分產能移到Flash，因此Flash在營收及出貨量都達歷史新高，且華邦電持續在Nor Flash是全球最大供應商。此外，Nor Flash月產能從年初的54000片成長到57000片，產能一直滿載，盡力把交期縮短。
隨著DRAM 25奈米投產增加，在第4季占所有DRAM出貨的20%，占DRAM總投片量的一半，且目前良率已達到業界水準，將會持續增加投產；且預期在高雄廠完成後，將挹注更多產能。目前高雄廠進度符合預期，明年上半年將機台移入和試產，明年下半年產出，主要生產25奈米的微縮版D25-S。
此外，有關近期德州大風雪對Nor Flash產能影響，華邦是否從中受惠？陳沛銘表示，在該地區英飛凌有很大的產能做nor flash，且大部分是提供給車用，雖然競爭對手可能從中獲得好處，但由於整體車用產能就很緊，一時間也沒有多餘產能補上，不會立即受惠；以華邦而言，DRAM提供車用部分較多，N

▲法人認為REITs將展開補漲行情。（圖／pixabay）
記者陳心怡／台北報導
此外，分析師將美國REITs在2021年整體獲利成長率上修至6.5%，目前P/NAV折價 5%，股價仍屬偏低，可望吸引長線買盤或法人機構加碼。
----------------------------------------------------------------------

[11] 新年手機也要保平安！電信業者推免費「手機健檢」

▲遠傳推出手機免費健檢服務。（圖／取自免費圖庫pixabay）
記者陳俐穎／台北報導
遠傳電信今日公布一項新服務，自2月22日至2月26日止，在北、中、南共9家門市推出手機及平板免費健診服務，由專業工程師依手機或平板狀況，提供專業建議，新年除了身體之外，手機也能保健康。
遠傳表示，消費者只要持安卓手機至遠傳指定健診服務門市，即享有32到59項手機功能免費檢查，蘋果手機的用戶則提供37項手機功能免費檢查，若手機或平板無法現場處理需轉送維修中心，所產生的維修費用，遠傳也提供95折優惠。
遠傳也表示，消費者只要至遠傳9家指定門市完成健診服務，並於該門市打卡按讚，還可獲得精美小禮物。同時，為了讓消費者在疫情期間安心使用手機或維持手機亮麗的外觀，全台遠傳門市皆備有手機消毒機、酒精及鍍膜液。提供服務的9家門市北部為台北館前、台北西寧南、三重正義北。中部為新竹中正、清水中山、逢甲福星。南部為嘉義民生北、永康中華、高雄林森。
----------------------------------------------------------------------

[12] 新台幣豬羊變色！尾盤升值變貶值　收在28.338元

▲新台幣收盤收28.338元。（圖／記者湯興漢攝）
記者陳心怡／台北報導
台北外匯市場新台幣兌美元匯率今（19）日收盤收28.338元，貶值0.8分。
新台幣今日以28.15元兌1美元開出，升值1.8角，隨著熱錢湧入，新台幣開盤不久隨即勁揚4.05角，來到27.925元，再度見到27字頭，不過終場在中央銀行調節下，升幅才顯著收斂，新台幣收在28.338元。
新台幣兌美元盤中最高28.338元，最低27.907元兌1美元。
---------------------------------------------------------

▲30歲男子初四選當天號碼刮中100萬，連樹蟾也來報喜。（示意圖／記者呂佳賢攝）
記者陳依旻／台北報導
樹蟾瞬間鳴叫報喜，30歲男性在台中市龍井區「抱發富彩券行」刮到100萬，台灣彩券公司今（19）日轉述彩券業者說法，中獎人每年過年都會買彩券、刮刮樂，當天到店裡有點猶豫要買哪張，於是問：「今天是幾號啊？」，店員回：「15號初四」，中獎人就選15號「2,000萬超級紅包」。
過了沒幾分鐘，客人就興高采烈的拿著刮刮樂到櫃檯，店員用投注機幫客人對獎，就在對獎的同時，店裡養的樹蟾也在瞬間鳴叫，像是「發！發！發」一樣，店員一看發現是中了100萬，這時看到客人十分驚訝，店員趕緊告訴他：「是真的有中大獎喔！恭喜啊！」。
客人才回過神說：「我還以為我眼花看錯了」。之前店裡曾經開出今彩539頭獎，當時用投注機幫忙客人對獎時，店裡樹蟾也是瞬間大聲鳴叫，真的是知道中大獎在報喜呢。
----------------------------------------------------------------------

[26] 他準備高級洋酒拜年！女友白眼「紅包呢」...狂討5位數：全家都要

▲原PO已經準備高級禮盒了，女友卻還是要求他包紅包。示意圖，與本文無關。（圖／記者嚴云岑攝）
記者曾筠淇／綜合報導
當戀人交往到一定的階段後，就會去見彼此父母。一名男網友發文表示，他日前特意買了洋酒高級禮盒要去女友家拜年，想不到對方卻質問他「那紅包呢？」更誇張的是，女友竟還說「她家每人都要包」，並詳列金額，總計高達4萬4600元。貼文曝光後，立刻引發網友熱議。
這名男網友在臉書粉專「靠北女友2.0」上發文表示，因為要去女友家拜年，所以對方事前就詢問他「準備好了嗎？」他聽聞後，立刻自信的回應自己已準備洋酒高級禮盒要當伴手禮，怎料此時，女友竟反問「那紅包呢？」
原PO當下覺得有點奇怪，因此提出疑問再次確認，豈料卻遭白眼伺候，得到一句「不然呢？」女友表示，原PO總共應該要準備6包，「她家每人都要包」，父母各要8800元、姊姊和弟弟各6600元，至於姊姊的2個小孩則各要3600元，而她本人「包6600元即可。」
▲女友要求的紅包總金額高達4萬4600元。（圖／翻攝自Facebook／靠北女友2.0）
原PO起初想著禮多人不怪，但聽到女友的要求後又覺得很疑惑，拜年有需要所有人都包紅包嗎？而且還限定金額，

▲示意圖，與本文無關。（圖／記者施怡妏攝）
網搜小組／周亭瑋報導
一名台大生日前在網路上，看到校友準備進台積電工作，沒想到翻開對方學歷，驚呆直呼「會不會浪費了？」PO文釣出不少同業，直言該職缺已經算食物鏈高層；但也有人建議，累積一下經驗，可以往外商公司跳。
台大生在dcard發文提到，「網路上偶然看到一個人說，他將去台積當產品工程師，點進去一看，交大電機學士、台大電機碩士，這樣學歷去台積電當產品工程師，會不會有點浪費了？」
對此，理工生紛紛回應，「還好吧，G產品（台積電產品工程師）也算食物鏈高層了，比產線三寶都好太多了」、「看透豬屎屋（IC設計公司），不少晚上也得加班，去GG product當食物鏈上層，客戶之下，萬人之上也不錯」、「根本夢幻工作了，哪裡浪費呀」、「產品已經是食物鏈頂端了，跟FA同一級，有些RD還比產品操」、「產品雖然都常日班、不用輪班，但並沒有很涼，對到水果公司產品線，平均保底9點後下班」。
▲示意圖，與本文無關。（圖／取自免費圖庫Pakutaso）
另一派則認為，「太浪費聰明才智了，應該拿個phd學位，再進去直接變33，可以少奮鬥好幾年，可惜呀可惜」、「過水累積一下經驗，就可以準備離開去外商了」；但也有老鳥直言，業界並不乏高學歷人才，「四大電類學碩在產品根本很多，連整合隨便抓都一堆這種背景的」、「不會，你以為這個學歷有多屌？甚至不是純的呢」。
說到產品工程師，其工作項目繁雜，主要任務就是確保每個生產環節緊緊相扣，流程順暢，從產品的設計、生產、測試到封裝都要了解，若能進一步拓展格局，有機會往更高職位晉升。
另外，有別於製程工程師，只負責一個製程，但不分產品；產品工程師通常負責一個產品的全部製程。不過，基本上兩者任務是差不多的。
►青春版的姐妹悄悄話《八年級生》沒聽過就老啦
----------------------------------------------------------------------

[36] 比特幣衝5萬美元！醫曝現處於「恐怖平衡」：沒有永不破的泡泡

▲比特幣已衝破5萬美元大關。（圖／免費圖庫pakutaso）
記者林妤柔／綜合報導
自從特斯拉執行長馬斯克（Elon Musk）買進價值15億美元的比特幣後，比特幣不斷創新高，18日更是衝破5.2萬美元，在過去一個月內上漲46%。急診兒科醫師認為，現在的狀況很像「恐怖

▲原PO幸運刮中100萬元。示意圖，與本文無關。（圖／記者林敬旻攝）
記者曾筠淇／綜合報導
許多民眾都趁著過年期間買刮刮樂試手氣，一名女網友發文表示，她日前幸運的刮中100萬元後，就被老闆「討吉利」，要求她分紅；不過，畢竟難得中大獎，所以她就好奇，應該要包多少錢給對方呢？貼文一出，立刻引發熱議，網友也留言回應，「我刮不中店家會包安慰獎給我嗎」、「店家沒理由討吃紅啊」。
▲老闆要求原PO分紅。示意圖，與本文無關。（圖／記者劉維榛攝）
原PO也在留言處提到，雖然家人告訴她其實可以不用包紅包給店家，但她中獎的當下，老闆就激動的要求加LINE，讓她擔心如果沒包，可能就會一直收到對方訊息。
貼文一出，網友紛紛留言回應，「就看你個人吧！錢是你的，他們也不能幹嘛」、「這種事情本來就沒有一定要」、「為什麼要包？覺得店家趁機敲竹槓，但是已經加LINE又蠻尷尬的」、「這種的絕對不要包，還主動跟你要真的很不要臉」、「捐出去比較值得吧？下次沒刮中記得跟老闆要安慰獎」。
原PO而後也更新貼文說明，畢竟中大獎，家人後來覺得其實可以分分喜氣，因此最後決定包給老闆6000元、員工一人2000元。
----------------------------------------------------------------------

[45] 智擎胰臟癌新藥取得5.6億里程碑金　股價飆漲停

▲智擎總經理王宏仁。（圖／智擎提供）
記者姚惠茹／台北報導
智擎（4162）昨（18）日公告開發的胰腺癌新藥安能得（ONIVYDE），在歐亞地區2020年度淨銷售額已達到第一期銷售里程碑，公司將收取2,000萬美元（約新台幣5.6億元）銷售里程金，可望貢獻第1季稅後純益約3.8元，推升今（19）日股價開盤即攻上漲停。
智擎表示，新藥ONIVYDE是授權給歐洲公司，再轉給日本公司代銷售，並在2020年3月取得日本上市許可，第2季底開始銷售，目前日本營收比重已佔歐亞市場的4成，這次銷售里程碑授權金主要就是來自日本市場貢獻。
智擎指出，新藥ONIVYDE自2011年經美國FDA及歐盟EMA認定為治療胰臟癌的罕見疾病藥物，透過授權合作夥伴法國Ipsen公司的推廣，已取得台灣、美國、歐洲、日本等共40多個國家的上市許可，使用於胰腺癌病人在接受過標準藥物gemcitabine失敗後的治療，同時正開發更多項目。
根據2

▲雲朗觀光集團宣佈2021年將以「零資產、純管理」模式優化獲利能力。（圖／雲品提供）
記者林淑慧／台北報導
後疫情時代，飯店業積極轉型優化營運。雲品國際今（18）日召開董事會，會中決議於第二季結束頤品大飯店新店北新館的宴會營運；雲品表示，今年宴會事業將持續透過零資產、純管理的經營模式，將集團餐飲優勢加以延伸，以期優化獲利能力。
去年以來受到新冠肺炎疫情衝擊，飯店產業受到喜宴、尾牙取消或遞延，宴會接單場次減少，另也因疫情關係影響館外餐飲業務，晶華、六福、國賓、亞都麗緻等均開發冷凍食品、外帶餐飲，或啟動資產開發計畫，以期達到更佳的營運績效。
據了解，去年迄今政府針對觀光產業祭出政策補貼，雲品國際雖符合申請條件，但執行長張安平指示將官方補貼留給更需要的同業，旗下飯店員工不僅沒有減薪，也沒有放無薪假，資方以集團資源、開啟新營運模式來支持自家飯店營運之虧損。
▲雲朗觀光集團宣佈2021年將以「零資產、純管理」模式優化獲利能力。（圖／雲品提供）
雲品國際表示，4月30日結束頤品大飯店新店北新館的宴會營運後，雲品將提供全數同仁於集團中轉職之機會，該館全數職工，包括廚師、服務生、業務人員等，將可選擇轉調至集團各適合營業據點。至於已預訂宴會或工商活動的客戶權益將不受影響。
另外，2月至4月間頤品大飯店新店北新館仍正常營業，持續為顧客服務。雲品國際與雲朗觀光集團未來將透過更多資源整合，讓餐飲宴會與住房業務持續發展，為各種需求顧客提供更多元服務。
展望後市，雲品國際表示，自從啟動君品Collection宴會平台後，成功以零資產模式與提供場地的業者共享利潤，降低場地固定租金負擔，以皇家薇庭和淡水嘉廬為例，在受疫情影響的2020年營業收入還優於2019年，是表現相當亮眼的營業模式，將持續推廣此類零資產成功營運的經驗，致力成為宴會事業產業鏈的整合者。
----------------------------------------------------------------------

[58] 快訊／元大投信總經理黃昭棠突請辭　公司回應是健康及家庭因素

▲元大投信總經理黃昭棠 。（圖／資料照）
記者周康玉／台北報導
元大投信總經理黃昭棠突然請辭，公司回應，是因為家庭及健康因素請辭，擬提報元大投信董事會後於110年2月23日起生效。
黃昭棠在投信圈資歷很深，早期，是在寶來投信任職，負責

▲吉爾以「咆哮小貓」（Roaring Kitty）為名開設頻道，引起GameStop軋空之戰。（圖／翻攝自YouTube）
記者林妤柔／綜合報導
GameStop軋空之戰的幕後推手吉爾（Keith Gill）週二在麻州被提起集體訴訟，指控他為了操縱市場，在Youtube、Twitter和Reddit宣傳GameStop股票，藉此牟取暴利。
此次訴訟由證券集體訴訟公司Hagens Berman Sobol Shapiro代表華盛頓州的Christian Iovin和處境類似的個人所提起。Iovin在股價低於100美元時，賣出價值20萬美元的GameStop股票買入選擇權合約，但不久就飆到每股超過400美元，迫使他以高價購回股票。
針對訴訟問題，吉爾表示，先前在金融業的工作從未涉及證券交易，或者為客戶提供諮詢服務，他堅信該公司的未來潛力，否認操縱一事。目前仍不清楚他在GameStop股票中賺多少錢，但該股在12月達到20美元時，他跟家人已經成為百萬富翁，即使後來飆漲的漲幅已經消失，獲利仍相當可觀。
----------------------------------------------------------------------

[68] Messenger有偷讀功能了！　長按對話框訊息全曝光

▲臉書也有偷讀訊息功能。（圖／翻攝自Messenger）
記者林妤柔／綜合報導
有時不想回別人的Line，可先「長按」對話框偷讀訊息，相信這個功能對於iPhone用戶來說並不陌生。先前Messenger沒有這個功能，但臉書最近也默默跟進，只要長按未讀取的對話框，就能提前先看到內容，對於怕已讀他人的用戶來說更加方便。
根據《ETtoday新聞雲》記者實測，只要常按Messenger的未讀取訊息，就會跑出有對話內容的小視窗，即使跳出來後，也不會顯示「已讀」，等到工作忙完、有心情再回訊息時，就不會有壓力或罪惡感。
不過，這個福利只有iPhone獨有，記者同樣實測Android手機，發現還沒有「偷看功能」，如果Android用戶想長按偷看Line或Messenger，可能需要再等等了。
----------------------------------------------------------------------

[69] 台新金併保德信案准駁兩關鍵　金管會：資

▲兒科急診醫師認為現在投資市場太瘋狂。（圖／記者張一中攝）
記者林妤柔／綜合報導
iOS用戶限定的線上語音交流App「Clubhouse」在台灣迅速竄紅，用戶可透過開聊天房間，有兒科急診醫師分享在聽過幾個股票房間後，深深覺得「整個市場太瘋狂」。
謝宗學指出，許多沒經歷過大空頭或剛開戶的人紛紛進場，雖然只有聲音，但對股市美好幻想的熱情癡迷，充斥整個Clubhouse房間，也讓他想起小時候看到小學老師在學校拿著收音機聽股市報價和解盤，那個台灣錢淹腳目的年代。
謝宗學認為，雖然錢放在銀行不會變多，但絕對數目一定不會變少。要拿錢重壓股市去年三月後多的是機會，現在絕對不是全力一搏的最佳時機。
----------------------------------------------------------------------

[83] 黃金跌至2個月來新低點　美經濟復甦、美元升值成主因

記者吳珍儀／綜合報導
由於美國經濟復甦優於預期，提振美元及美國公債殖利率，連帶影響黃金作為避險工具的吸引力，4月黃金期貨在週三（17）日下跌1.46%至每盎司1772.8元，已是金價連續第五個交易日下跌，且跌至近兩個月來低點。
期貨公司High Ridge的貴金屬交易部門主管David Meger提到，由於對戰勝新冠疫情持樂觀情緒，美元因此小幅度升值，美國公債殖利率也上升至2020年2月以來水平。
截至稍早，4月黃金期貨價格上漲0.49%，至每盎司1781.4美元。
其他貴金屬部分，3月白銀期貨價格下跌0.20%至每盎司27.26美元，4月鉑金期貨價格上漲1.19%至每盎司1272.7美元；3月鈀金期貨價格上漲0.29%至每盎司2376.5美元。
----------------------------------------------------------------------

[84] 陸製特定鋁箔要被課「臨時反傾銷稅」！　財政部：最高稅率32.36%

▲財政部將對陸製特定鋁箔課臨時反傾銷稅。（圖／資料照）
記者陳依旻／台北報導
針對自中國大陸產製進口特定鋁箔反傾銷調查案，財政部關務署今（18）日表示，財政部及經濟部初步認定有傾銷情事，並對國內產業造成實質損害。
為防止我國產業在調查期間繼續遭受損害，財政部公告自110年2月22日起對上述產品臨時課徵反傾銷稅，為期4個月

▲三星S20 FE降價6500元。（圖／傑昇通信提供）
記者陳俐穎／台北報導
農曆春節剛過，大家紛紛拿壓歲錢或是年終獎金來升級裝備，各大手機品牌也紛紛對年後將要發表的新機上市摩拳擦掌，去年Q3上市的Galaxy S20 FE是三星主打「輕旗艦」規格的5G手機，因外媒報導接班人Galaxy S21 FE將於下半年度面世，價格開始往下落，18000元有找。
習慣在上半年推出Galaxy S系列旗艦機款，下半年發表向S系列致敬的Fan Edition旗艦產品的三星，2021年也是如法泡製，日前外媒釋出Galaxy S21 FE即將於下半年問世，去年5G高規格輕旗艦機款Galaxy S20 FE 5G則開始降價，準備在正式銜接Galaxy S21 FE 5G前，搶攻5G中階手機換機潮。
Galaxy S20 FE 5G搭載高通S865旗艦級處理器、6.5吋螢幕、大感光元件的三鏡頭主相機、3200萬畫素前相機、4500mAh電池與無線快充、無線電力分享等各種5G旗艦手機規格，當時23,990元的親民價格已造成旋風，北台灣最大連鎖傑昇通信官網顯示，三星Galaxy S20 FE 5G年後降6,500元來到17,490元。
除此之外，原本價格就不用萬元的4G中階手機三星Galaxy A31，擁有6.4吋Infinity-U螢幕、4,800萬畫素四鏡頭及5,000mAh電量的，也下殺3千至6,990元。
除此之外，面對年後各家手機品牌陸續推出新機，蘋果也開始降價搶市，去年10月上市的新機iPhone 12出現超過2千5的折扣，搭載A12仿生晶片，配置10.2吋顯示器，搭配立體聲音響系統，機身採用100%再生鋁金屬打造，具備防刮耐磨的特性，還能降低環境生態負擔的iPad也跟著降價促銷。


In [9]:
browser.close()